In [2]:
%pip install ipywidgets

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [3]:
%pip install numpy==1.23.5 torch laion_clap torchvision optuna einops opencv-python moviepy

Defaulting to user installation because normal site-packages is not writeable
Note: you may need to restart the kernel to use updated packages.


In [4]:
import os
from tqdm import tqdm
import optuna

import torch

import laion_clap
import librosa

from models.model import SimilarityRecognizer
from src.preprocess import Preprocess

import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score, f1_score

from src.serialization import read_obj, save_obj
from src.config import S3_MODEL_PATH, LOCAL_MODEL_PATH, BUCKET_URL
from src.utils import download_file_from_s3, mp4, calculate_similarities
from src.video_analysis import get_video_features
from src.audio_analysis import get_audio_features


In [5]:
device = 'cuda' if torch.cuda.is_available() else 'cpu'

In [6]:
if not os.path.exists(LOCAL_MODEL_PATH):
    download_file_from_s3(BUCKET_URL, S3_MODEL_PATH, LOCAL_MODEL_PATH)

In [7]:
video_model = SimilarityRecognizer(model_type="base", batch_size=8).to(device)
video_model.load_pretrained_weights("checkpoints/best_model_base_224_16x16_rgb.pth")
video_model.eval()

SimilarityRecognizer(
  (feature_extractor): Timesformer(
    (model): VisionTransformer(
      (dropout): Dropout(p=0.0, inplace=False)
      (patch_embed): PatchEmbed(
        (proj): Conv2d(3, 768, kernel_size=(16, 16), stride=(16, 16))
      )
      (pos_drop): Dropout(p=0.0, inplace=False)
      (time_drop): Dropout(p=0.0, inplace=False)
      (blocks): ModuleList(
        (0): Block(
          (norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
          (attn): Attention(
            (qkv): Linear(in_features=768, out_features=2304, bias=True)
            (proj): Linear(in_features=768, out_features=768, bias=True)
            (proj_drop): Dropout(p=0.0, inplace=False)
            (attn_drop): Dropout(p=0.0, inplace=False)
          )
          (temporal_norm1): LayerNorm((768,), eps=1e-06, elementwise_affine=True)
          (temporal_attn): Attention(
            (qkv): Linear(in_features=768, out_features=2304, bias=True)
            (proj): Linear(in_features=768, 

In [8]:
audio_model = laion_clap.CLAP_Module(enable_fusion=False, device=device, amodel='HTSAT-base')
audio_model.load_ckpt('checkpoints/music_speech_audioset_epoch_15_esc_89.98.pt') 

  return _VF.meshgrid(tensors, **kwargs)  # type: ignore[attr-defined]

Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['roberta.pooler.dense.bias', 'roberta.pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
  warnings.warn(

  checkpoint = torch.load(checkpoint_path, map_location=map_location)



Load the specified checkpoint checkpoints/music_speech_audioset_epoch_15_esc_89.98.pt from users.
Load Checkpoint...
logit_scale_a 	 Loaded
logit_scale_t 	 Loaded
audio_branch.spectrogram_extractor.stft.conv_real.weight 	 Loaded
audio_branch.spectrogram_extractor.stft.conv_imag.weight 	 Loaded
audio_branch.logmel_extractor.melW 	 Loaded
audio_branch.bn0.weight 	 Loaded
audio_branch.bn0.bias 	 Loaded
audio_branch.patch_embed.proj.weight 	 Loaded
audio_branch.patch_embed.proj.bias 	 Loaded
audio_branch.patch_embed.norm.weight 	 Loaded
audio_branch.patch_embed.norm.bias 	 Loaded
audio_branch.layers.0.blocks.0.norm1.weight 	 Loaded
audio_branch.layers.0.blocks.0.norm1.bias 	 Loaded
audio_branch.layers.0.blocks.0.attn.relative_position_bias_table 	 Loaded
audio_branch.layers.0.blocks.0.attn.qkv.weight 	 Loaded
audio_branch.layers.0.blocks.0.attn.qkv.bias 	 Loaded
audio_branch.layers.0.blocks.0.attn.proj.weight 	 Loaded
audio_branch.layers.0.blocks.0.attn.proj.bias 	 Loaded
audio_branch.laye

In [9]:
data_path = os.path.join(os.getcwd(), 'train_data_yappy', 'train_dataset')
train = pd.read_csv('train_data_yappy/train.csv')
train['created'] = pd.to_datetime(train['created'])
train


,created,uuid,link,is_duplicate,duplicate_for,is_hard
0,2024-06-01 00:05:43,23fac2f2-7f00-48cb-b3ac-aac8caa3b6b4,https://s3.ritm.media/yappy-db-duplicates/23fa...,False,NaN,False
1,2024-06-01 00:11:01,2fa37210-3c25-4a87-88f2-1242c2c8a699,https://s3.ritm.media/yappy-db-duplicates/2fa3...,False,NaN,False
2,2024-06-01 00:13:20,31cc33d5-95de-4799-ad01-87c8498d1bde,https://s3.ritm.media/yappy-db-duplicates/31cc...,False,NaN,False
3,2024-06-01 00:27:23,03abd0ec-609e-4eea-9f2a-b6b7442bc881,https://s3.ritm.media/yappy-db-duplicates/03ab...,False,NaN,False
4,2024-06-01 00:30:23,22ee0045-004b-4c7e-98f2-77e5e02e2f15,https://s3.ritm.media/yappy-db-duplicates/22ee...,False,NaN,False
...,...,...,...,...,...,...
22758,2024-09-12 13:46:57,0efe756a-e965-40c1-94db-de7f3e6649a9,https://s3.ritm.media/yappy-db-duplicates/0efe...,True,131846f3-6f5c-497a-a2fa-95cfb3929301,False
22759,2024-09-12 14:46:13,caec3b94-e356-4576-b00a-515e0df1dfc3,https://s3.ritm.media/yappy-db-duplicates/caec...,True,3b5eb15a-c6d7-4214-8dd6-c029564ff11d,False
22760,2024-09-13 09:08:42,c5b69151-f240-4e27-a5c9-c41f79a167e9,https://s3.ritm.media/yappy-db-duplicates/c5b6...,True,17ecc94a-f28a-40d5-b438-86b6e82a2fef,False
22761,2024-09-13 14:52:21,6d3233b6-f8de-49ba-8697-bb30dbf825f7,https://s3.ritm.media/yappy-db-duplicates/6d32...,True,1838f7a7-ef2a-4141-a125-90fb5bf0c5a2,False


In [10]:
train_files = set(map(lambda x: x.split('.')[0], os.listdir(data_path)))

In [11]:
filtred_train = train[train['uuid'].isin(train_files)].sort_values(by='created')

In [12]:
filtred_train

,created,uuid,link,is_duplicate,duplicate_for,is_hard
0,2024-06-01 00:05:43,23fac2f2-7f00-48cb-b3ac-aac8caa3b6b4,https://s3.ritm.media/yappy-db-duplicates/23fa...,False,NaN,False
1,2024-06-01 00:11:01,2fa37210-3c25-4a87-88f2-1242c2c8a699,https://s3.ritm.media/yappy-db-duplicates/2fa3...,False,NaN,False
2,2024-06-01 00:13:20,31cc33d5-95de-4799-ad01-87c8498d1bde,https://s3.ritm.media/yappy-db-duplicates/31cc...,False,NaN,False
3,2024-06-01 00:27:23,03abd0ec-609e-4eea-9f2a-b6b7442bc881,https://s3.ritm.media/yappy-db-duplicates/03ab...,False,NaN,False
4,2024-06-01 00:30:23,22ee0045-004b-4c7e-98f2-77e5e02e2f15,https://s3.ritm.media/yappy-db-duplicates/22ee...,False,NaN,False
...,...,...,...,...,...,...
10253,2024-07-07 09:50:11,67457d6c-6175-400d-89fe-7243d73b7874,https://s3.ritm.media/yappy-db-duplicates/6745...,True,44bec7ea-5636-44d6-866c-7cdd7f9a4696,False
10290,2024-07-07 13:38:24,f0968632-e0b3-4407-9c99-3258e75e2959,https://s3.ritm.media/yappy-db-duplicates/f096...,True,3fb52c06-65ae-4ea6-9f57-c4b3f9ac7163,False
10543,2024-07-08 10:32:40,7282037d-678f-4397-b6c7-c0c8819533da,https://s3.ritm.media/yappy-db-duplicates/7282...,True,03f52b57-5e60-441f-8c41-b78f9222752e,False
10690,2024-07-08 22:21:52,6e85101c-da03-4416-89f1-0284da3bb478,https://s3.ritm.media/yappy-db-duplicates/6e85...,True,455d9167-5327-45b7-832a-249ca1354778,False


In [13]:
duplicates_df = filtred_train[~filtred_train['duplicate_for'].isna()]
duplicates_df

,created,uuid,link,is_duplicate,duplicate_for,is_hard
97,2024-06-01 10:27:10,5eb4127e-5694-492b-963c-6688522e9ad2,https://s3.ritm.media/yappy-db-duplicates/5eb4...,True,3726bb2d-3323-41f8-8eb2-0d7cf095d62b,False
181,2024-06-01 19:09:52,b5f191e6-42e0-43f5-8773-560643de17fb,https://s3.ritm.media/yappy-db-duplicates/b5f1...,True,314d2988-eb85-4581-8416-da998e036afe,False
195,2024-06-01 20:53:41,025ee26a-7391-4f60-878a-7fc1928a967b,https://s3.ritm.media/yappy-db-duplicates/025e...,True,0fd09c1b-e19e-4b6e-84e6-35f9d4fc6f72,False
212,2024-06-01 22:04:03,a18324cf-b2ad-41e2-86b8-e6923c5fdc36,https://s3.ritm.media/yappy-db-duplicates/a183...,True,2da574f2-5ae7-4d85-9f5b-c2970a315c1c,False
221,2024-06-01 23:12:31,2253aaa4-b29c-4b7d-b9cc-9286d23c44e8,https://s3.ritm.media/yappy-db-duplicates/2253...,True,3a20cbaf-798a-4b25-b2c1-e41e1c6ecd09,False
...,...,...,...,...,...,...
10253,2024-07-07 09:50:11,67457d6c-6175-400d-89fe-7243d73b7874,https://s3.ritm.media/yappy-db-duplicates/6745...,True,44bec7ea-5636-44d6-866c-7cdd7f9a4696,False
10290,2024-07-07 13:38:24,f0968632-e0b3-4407-9c99-3258e75e2959,https://s3.ritm.media/yappy-db-duplicates/f096...,True,3fb52c06-65ae-4ea6-9f57-c4b3f9ac7163,False
10543,2024-07-08 10:32:40,7282037d-678f-4397-b6c7-c0c8819533da,https://s3.ritm.media/yappy-db-duplicates/7282...,True,03f52b57-5e60-441f-8c41-b78f9222752e,False
10690,2024-07-08 22:21:52,6e85101c-da03-4416-89f1-0284da3bb478,https://s3.ritm.media/yappy-db-duplicates/6e85...,True,455d9167-5327-45b7-832a-249ca1354778,False


In [14]:
len(duplicates_df)

469

In [15]:
filtred_train[filtred_train['is_hard']]

,created,uuid,link,is_duplicate,duplicate_for,is_hard
8,2024-06-01 00:41:33,1e9efc51-a74c-4f32-b03e-71905f8d6dd1,https://s3.ritm.media/yappy-db-duplicates/1e9e...,False,NaN,True
77,2024-06-01 08:28:26,3dd424ce-f0e5-4727-88a3-8f318b612afd,https://s3.ritm.media/yappy-db-duplicates/3dd4...,False,NaN,True
211,2024-06-01 22:03:27,d444f2d0-a7cd-4c9b-bc56-8d5ef88ec015,https://s3.ritm.media/yappy-db-duplicates/d444...,False,NaN,True
356,2024-06-02 11:23:19,49292bf9-dc53-44d5-9980-a658ab3a3921,https://s3.ritm.media/yappy-db-duplicates/4929...,False,NaN,True
472,2024-06-02 20:54:43,6a816304-7c1b-4c16-81c6-09f46bb0ad63,https://s3.ritm.media/yappy-db-duplicates/6a81...,False,NaN,True
553,2024-06-03 04:13:06,1f4df893-663e-4ca3-a6ca-46819aebd8a3,https://s3.ritm.media/yappy-db-duplicates/1f4d...,False,NaN,True
581,2024-06-03 06:43:43,f41954d9-dd9f-4844-bea4-4a86b5ce2ae9,https://s3.ritm.media/yappy-db-duplicates/f419...,False,NaN,True
748,2024-06-03 21:05:43,992321a1-06b2-4f2c-a2d1-aa7f37a14da1,https://s3.ritm.media/yappy-db-duplicates/9923...,False,NaN,True
939,2024-06-04 13:11:37,68e41e6f-2d9b-409d-a3b8-581dff56a722,https://s3.ritm.media/yappy-db-duplicates/68e4...,False,NaN,True
952,2024-06-04 13:53:38,bf2c9cd9-6fd2-4afd-bd90-b2a5d132cd52,https://s3.ritm.media/yappy-db-duplicates/bf2c...,False,NaN,True


In [16]:
# '5eb4127e-5694-492b-963c-6688522e9ad2' in all_features 

In [17]:
video_names = filtred_train['uuid'].to_list()

video_paths = [os.path.join(data_path, mp4(name)) for name in video_names]

In [18]:
# from tqdm import tqdm
# import torch
# import librosa
# import os
# from moviepy.editor import VideoFileClip
# import numpy as np

# def get_audio_features(model, features=None, video_folder=None, video_paths=None):
#     if video_folder is not None:
#         video_paths = [os.path.join(video_folder, name) for name in os.listdir(video_folder)]
#     elif video_paths is None:
#         raise ValueError("Either video_folder or video_paths must be provided")

#     print("Starting to extract audio features....")    


#     if features:
#         all_features = features
#     else:
#         all_features = {}

#     for path in tqdm(video_paths, ncols=70):
#         video_id = os.path.basename(path.split('.')[0])

#         if video_id in all_features:
#             continue   
        
#         video = VideoFileClip(path)
#         audio = video.audio

#         if audio is None:
#             continue

#         audio_data = audio.to_soundarray(fps=48000)
#         audio_data = np.mean(audio_data, axis=1)  # Преобразуем в моно, если требуется
#         # audio_data = audio_data[:, 0]
        
#         # audio_data, _ = librosa.load(audio_data, sr=48000) # sample rate should be 48000
#         audio_data = audio_data.reshape(1, -1) # Make it (1,T) or (N,T)

#         with torch.no_grad():
#             audio_embed = model.get_audio_embedding_from_data(x = audio_data, use_tensor=False)

#         all_features[video_id] = audio_embed

#     return all_features

In [19]:
audio_features = read_obj('pickled_data/audio_features_2500.pkl')

audio_features = get_audio_features(audio_model, features=audio_features, video_paths=video_paths)

# save_obj(audio_features, 'features_pickled/audio_features_2500')

Starting to extract audio features....


100%|███████████████████████████| 2500/2500 [00:00<00:00, 3069.55it/s]


In [20]:
preprocess = Preprocess(clip_len=8, out_size=224, frame_interval=1, channels=1)

video_features = read_obj('pickled_data/grey_features_2500.pkl')

video_features = get_video_features(preprocess, video_model, features=video_features, video_paths=video_paths)

# save_obj(grey_features, 'features_pickled/grey_features_2500')

Starting to extract video features....


100%|█████████████████████████| 2500/2500 [00:00<00:00, 552813.16it/s]


In [29]:
list(audio_features.keys())[0]

'23fac2f2-7f00-48cb-b3ac-aac8caa3b6b4'

In [32]:
video_features[list(video_features.keys())[0]].shape

torch.Size([1, 768])

In [39]:
audio_features_new = audio_features

In [40]:
audio_features_new = {key: torch.tensor(emb) for key, emb in audio_features_new.items()}

In [37]:
for key, emb in video_features.items():
    print(key, emb.shape)
    break

23fac2f2-7f00-48cb-b3ac-aac8caa3b6b4 torch.Size([1, 768])


In [ ]:
for key, emb in video_features.items():
    print(key, torch.tensor(emb))
    break

In [38]:
for key, emb in audio_features.items():
    print(key, torch.tensor(emb).shape)
    break

23fac2f2-7f00-48cb-b3ac-aac8caa3b6b4 torch.Size([1, 512])


In [31]:
audio_features[list(audio_features.keys())[0]].shape

(1, 512)

In [44]:
original_name = '1e9efc51-a74c-4f32-b03e-71905f8d6dd1'
compare_name = '68e41e6f-2d9b-409d-a3b8-581dff56a722'

In [45]:
from moviepy.editor import VideoFileClip

for name in [original_name, compare_name]:
    video = VideoFileClip(f'train_data_yappy/train_dataset/{name}.mp4')
    audio = video.audio
    audio.write_audiofile(f'audios/{name}.wav')

MoviePy - Writing audio in audios/1e9efc51-a74c-4f32-b03e-71905f8d6dd1.wav


MoviePy - Done.
MoviePy - Writing audio in audios/68e41e6f-2d9b-409d-a3b8-581dff56a722.wav


MoviePy - Done.


In [47]:
from audio_similarity.audio_similarity.audio_similarity import AudioSimilarity

# Paths to the original and compariosn audio files/folders

original_path = f'audios/{original_name}.wav'
compare_path = f'audios/{compare_name}.wav'

# Set the sample rate and weights for the metrics

sample_rate = 44100
weights = {
    'zcr_similarity': 0,
    'rhythm_similarity': 0.25,
    'chroma_similarity': 0.25,
    'spectral_contrast_similarity': 0.15,
    'perceptual_similarity': 0.25
}

sample_size = 20 # If you have a large number of audio files, its much faster to select a randome sample from each

verbose = True # Show logs

# Create an instance of the AudioSimilarity class

audio_similarity = AudioSimilarity(original_path, compare_path, sample_rate, weights, verbose=verbose, sample_size=sample_size)

# Calculate a single metric

zcr_similarity = audio_similarity.zcr_similarity()

# Calculate the Stent Weighted Audio Similarity

similarity_score = audio_similarity.stent_weighted_audio_similarity(metrics='all') # You can select all metrics or just the 'swass' metric

print(f"Stent Weighted Audio Similarity: {similarity_score}")

# print(zcr_similarity)

Loading comparison files:: 100%|████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████████| 1/1 [00:00<00:00,  3.35it/s]


Stent Weighted Audio Similarity: {'zcr_similarity': 0.9881230736755325, 'rhythm_similarity': 0.5221554095422234, 'chroma_similarity': 0.7234279215335846, 'spectral_contrast_similarity': 0.8824501339642326, 'perceptual_similarity': 0.5564428095141841, 'swass': 0.5828740552421329}


In [41]:
hard_similarities = calculate_similarities(video_id='1e9efc51-a74c-4f32-b03e-71905f8d6dd1', all_features=audio_features_new)
sorted_dict = dict(sorted(hard_similarities.items(), key=lambda item: -item[1]))
print(sorted_dict)

{'68e41e6f-2d9b-409d-a3b8-581dff56a722': 0.9620751738548279, 'd444f2d0-a7cd-4c9b-bc56-8d5ef88ec015': 0.9569374918937683, '2cf8f595-acd3-489f-b4ca-86d02c4e1eb2': 0.9452430605888367, '6a816304-7c1b-4c16-81c6-09f46bb0ad63': 0.9361286759376526, '3aa52ed7-f04c-43dc-8b1c-409fb61fdb16': 0.9350016713142395, '4c5b873f-eaa2-4f88-b9f2-e30a5c92f093': 0.9338694214820862, '416650c0-1ffd-41fd-b5eb-5b431517fb59': 0.9269060492515564, '474333ac-c084-46e5-9de3-8894222ac39a': 0.922494649887085, '2fa1b683-46b4-4c3b-826b-49b47f7c448d': 0.9187702536582947, '0268ec6a-e40d-4398-980d-d277db377b91': 0.9179416298866272, '28ad5c7f-c17c-4eba-8508-ff785b67c509': 0.9130191802978516, '390f73b5-6e10-4db6-978d-03cf06d2cc39': 0.910189151763916, '36e8015f-582e-4373-831c-2237ca1bbaba': 0.9089893102645874, '0590ee95-5247-4beb-8726-62b8e032a9b9': 0.9060426950454712, '8a894e66-f53f-46af-aed3-28ca8e2bd7f1': 0.8953107595443726, '3dd424ce-f0e5-4727-88a3-8f318b612afd': 0.8949084877967834, '0e3b1e0e-0bcd-4e50-aecb-648ab45628aa': 0

In [42]:
hard_similarities = calculate_similarities(video_id='1e9efc51-a74c-4f32-b03e-71905f8d6dd1', all_features=video_features)
sorted_dict = dict(sorted(hard_similarities.items(), key=lambda item: -item[1]))
print(sorted_dict)

{'68e41e6f-2d9b-409d-a3b8-581dff56a722': 0.9475602507591248, '6a816304-7c1b-4c16-81c6-09f46bb0ad63': 0.891708493232727, 'd444f2d0-a7cd-4c9b-bc56-8d5ef88ec015': 0.804998517036438, '742c2897-b1c7-48ab-9751-7ec7adc5640b': 0.7329480051994324, '2d6a4f5b-0366-4111-852e-5ad85a16406e': 0.7319097518920898, '0067b0d6-ba3b-40fb-8535-3cb4809f8d07': 0.711877703666687, 'd96be0a4-2224-4e11-b182-5cde82c0a386': 0.7074141502380371, '30b7e872-e912-47cc-a42e-a765623dbaa2': 0.704011857509613, '0c8102b9-c4a4-4a3c-9be4-a14dc665b9f4': 0.6947602033615112, '57c36c3c-bc36-45b4-9dea-6751997cf25d': 0.6861547231674194, '4ffc865e-fc3a-4566-a4a7-261635915dd9': 0.682123601436615, '22432439-32ab-4809-808e-0196fc9e0641': 0.6804153323173523, '4f93c2f2-7aea-4e55-89a3-836c1aa156bc': 0.6743459105491638, '174c1db2-aaa2-4e92-b9f8-157f4c2b3d86': 0.6733528971672058, '0c11b3bc-a75f-40b6-bba4-492895002c08': 0.672332763671875, '5534c695-f183-48c1-a9fd-266a83ed54e8': 0.6694144010543823, '0518bd8c-5647-41e4-970b-4f003f40336d': 0.668

In [26]:
for index, row in duplicates_df.iterrows():
    print(f'Similarity between {row['uuid']} and {row["duplicate_for"]}: {calculate_similarities(video_id=row['uuid'], 
                                                                                               second_video_id=row["duplicate_for"], 
                                                                                               all_features=video_features)}')

Similarity between 5eb4127e-5694-492b-963c-6688522e9ad2 and 3726bb2d-3323-41f8-8eb2-0d7cf095d62b: 0.9994394183158875
Similarity between b5f191e6-42e0-43f5-8773-560643de17fb and 314d2988-eb85-4581-8416-da998e036afe: 0.9997285008430481
Similarity between 025ee26a-7391-4f60-878a-7fc1928a967b and 0fd09c1b-e19e-4b6e-84e6-35f9d4fc6f72: 0.9932383894920349
Similarity between a18324cf-b2ad-41e2-86b8-e6923c5fdc36 and 2da574f2-5ae7-4d85-9f5b-c2970a315c1c: 0.8075734376907349
Similarity between 2253aaa4-b29c-4b7d-b9cc-9286d23c44e8 and 3a20cbaf-798a-4b25-b2c1-e41e1c6ecd09: 0.9990720748901367
Similarity between 99c8cd59-5995-4981-8346-460d40e4eed3 and 1a713f01-e2e1-48e8-83f3-2c90938ba197: 0.9964303970336914
Similarity between 5acd8e68-99fe-43fd-b9f9-b2279bdc9372 and 0af145dd-e5f1-4cc1-a755-4e9a52ea2435: 0.9971468448638916
Similarity between 05d72fc9-89a3-47bf-bb1b-6db4fc6f2b56 and 090cb968-02eb-4aba-8b94-f6124050df59: 0.9905136823654175
Similarity between 36e972c3-7134-41ce-b37f-e227fece4575 and 57ee

In [32]:
filtred_train

,created,uuid,link,is_duplicate,duplicate_for,is_hard
0,2024-06-01 00:05:43,23fac2f2-7f00-48cb-b3ac-aac8caa3b6b4,https://s3.ritm.media/yappy-db-duplicates/23fa...,False,NaN,False
1,2024-06-01 00:11:01,2fa37210-3c25-4a87-88f2-1242c2c8a699,https://s3.ritm.media/yappy-db-duplicates/2fa3...,False,NaN,False
2,2024-06-01 00:13:20,31cc33d5-95de-4799-ad01-87c8498d1bde,https://s3.ritm.media/yappy-db-duplicates/31cc...,False,NaN,False
3,2024-06-01 00:27:23,03abd0ec-609e-4eea-9f2a-b6b7442bc881,https://s3.ritm.media/yappy-db-duplicates/03ab...,False,NaN,False
4,2024-06-01 00:30:23,22ee0045-004b-4c7e-98f2-77e5e02e2f15,https://s3.ritm.media/yappy-db-duplicates/22ee...,False,NaN,False
...,...,...,...,...,...,...
10253,2024-07-07 09:50:11,67457d6c-6175-400d-89fe-7243d73b7874,https://s3.ritm.media/yappy-db-duplicates/6745...,True,44bec7ea-5636-44d6-866c-7cdd7f9a4696,False
10290,2024-07-07 13:38:24,f0968632-e0b3-4407-9c99-3258e75e2959,https://s3.ritm.media/yappy-db-duplicates/f096...,True,3fb52c06-65ae-4ea6-9f57-c4b3f9ac7163,False
10543,2024-07-08 10:32:40,7282037d-678f-4397-b6c7-c0c8819533da,https://s3.ritm.media/yappy-db-duplicates/7282...,True,03f52b57-5e60-441f-8c41-b78f9222752e,False
10690,2024-07-08 22:21:52,6e85101c-da03-4416-89f1-0284da3bb478,https://s3.ritm.media/yappy-db-duplicates/6e85...,True,455d9167-5327-45b7-832a-249ca1354778,False


In [ ]:
all_similarities = calculate_similarities(all_features=all_features, model=model) 

save_obj(all_similarities, 'pickled_data/similarities')

In [ ]:
all_similarities = calculate_similarities(all_features=all_features, model=model) 

save_obj(all_similarities, 'pickled_data/similarities')

In [64]:
all_similarities = calculate_similarities(all_features=all_features, model=model) 

save_obj(all_similarities, 'pickled_data/similarities')

In [81]:
def objective(trial):
    similarity_threshold = trial.suggest_float("similarity_threshold", 0, 1)

    db_videos = []
    duplicate_videos = []

    y_true = list(map(str, filtred_train['duplicate_for'].to_list()))
    y_pred = []

    for index, row in tqdm(filtred_train.iterrows(), ncols=70, total=len(filtred_train)):
        new_video_id = row['uuid']

        max_similarity = float('-inf')
        max_similarity_video = None

        for old_video in db_videos:
            similarity = all_similarities[new_video_id][old_video]

            if similarity > max_similarity:
                max_similarity = similarity
                max_similarity_video = old_video

        if max_similarity < similarity_threshold:
            db_videos.append(new_video_id)
            y_pred.append('nan')
        else:
            duplicate_videos.append(new_video_id)
            y_pred.append(max_similarity_video)

    new_y_true, new_y_pred = [], []

    for true, pred in zip(y_true, y_pred):
        is_true_nan = true == 'nan'
        is_pred_nan = pred == 'nan'

        new_y_true.append(1 if not is_true_nan else 0)
        new_y_pred.append(1 if not is_pred_nan else 0)

        if not is_true_nan and not is_pred_nan:
            new_y_pred[-1] = 1 if true == pred else 0
            
    # accuracy = accuracy_score(y_true, y_pred)

    score = f1_score(new_y_true, new_y_pred)
    
    return score

study = optuna.create_study(direction="maximize")
study.optimize(objective, n_trials=100)

print("Best hyperparameters: ", study.best_params)
print("Best f1_score: ", study.best_value)

[I 2024-09-28 16:22:31,827] A new study created in memory with name: no-name-2601676b-9ce4-4c7a-857d-9b1664f750f5
100%|██████████████████████████| 2500/2500 [00:00<00:00, 14041.99it/s]
[I 2024-09-28 16:22:32,017] Trial 0 finished with value: 0.0008009611533840609 and parameters: {'similarity_threshold': 0.11758916501104155}. Best is trial 0 with value: 0.0008009611533840609.
100%|███████████████████████████| 2500/2500 [00:00<00:00, 6029.27it/s]
[I 2024-09-28 16:22:32,441] Trial 1 finished with value: 0.38275862068965516 and parameters: {'similarity_threshold': 0.9977618675154444}. Best is trial 1 with value: 0.38275862068965516.
100%|███████████████████████████| 2500/2500 [00:00<00:00, 7595.45it/s]
[I 2024-09-28 16:22:32,779] Trial 2 finished with value: 0.43422913719943423 and parameters: {'similarity_threshold': 0.6863327954868816}. Best is trial 2 with value: 0.43422913719943423.
100%|██████████████████████████| 2500/2500 [00:00<00:00, 16318.12it/s]
[I 2024-09-28 16:22:32,941] Trial

Best hyperparameters:  {'similarity_threshold': 0.8601731272679782}
Best f1_score:  0.96


In [ ]:
similarity_threshold = 0.8601731272679782

df = filtred_train[filtred_train['is_hard']]

db_videos = []
duplicate_videos = []

y_true = list(map(str, filtred_train['duplicate_for'].to_list()))
y_pred = []

for index, row in tqdm(filtred_train.iterrows(), ncols=70, total=len(filtred_train)):
    new_video_id = row['uuid']

    max_similarity = float('-inf')
    max_similarity_video = None

    for old_video in db_videos:
        similarity = all_similarities[new_video_id][old_video]

        if similarity > max_similarity:
            max_similarity = similarity
            max_similarity_video = old_video

    if max_similarity < similarity_threshold:
        db_videos.append(new_video_id)
        y_pred.append('nan')
    else:
        duplicate_videos.append(new_video_id)
        y_pred.append(max_similarity_video)

new_y_true, new_y_pred = [], []

for true, pred in zip(y_true, y_pred):
    is_true_nan = true == 'nan'
    is_pred_nan = pred == 'nan'

    new_y_true.append(1 if not is_true_nan else 0)
    new_y_pred.append(1 if not is_pred_nan else 0)

if not is_true_nan and not is_pred_nan:
    new_y_pred[-1] = 1 if true == pred else 0
    
# accuracy = accuracy_score(y_true, y_pred)

score = f1_score(new_y_true, new_y_pred)